In [1]:
# !pip install -q accelerate==0.21.0 transformers==4.31.0 xformers bitsandbytes==0.40.2  # peft==0.4.0
# !pip install einops  # for falcon model
# !pip install torch

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, logging, BitsAndBytesConfig
import torch, re

logging.set_verbosity(logging.CRITICAL)

In [3]:
llama2 = 'NousResearch/Llama-2-7b-chat-hf'
falcon7b = 'tiiuae/falcon-7b-instruct'

# Load the entire model on the GPU 0
DEVICE = {"": 0}  # or try 'auto'
print(DEVICE)

{'': 0}


In [4]:
def load_model(model_name):
    # NOTE: BitsAndBytesConfig might be unnecessary. I used it as an attempt to get falcon to load in colab.
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_use_double_quant=True,
    )

    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        low_cpu_mem_usage=True,  # Auto set to true when specifying device_map
        return_dict=True,
        torch_dtype=torch.float16,
        device_map=DEVICE,  # device_map
        trust_remote_code=True,
        quantization_config=bnb_config
        # load_in_4bit=True,  # unsure if this should maybe help with loading it
    )

    tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "right"
    
    return model, tokenizer

    
def predict_model(model, tokenizer, med_type, context):
    # Run text generation pipeline with our next model
    prompt = f"Write me a {med_type} meditation script which {GOALS[med_type]}"

    if model_name == llama2:
        tokenized_prompt = f"""<s>[INST] <<SYS>>\n{context}\n<</SYS>>\n\n{prompt} [/INST]"""
    elif model_name == falcon7b:
        # Do not have a stray space at end of it will mess with tokenization
        tokenized_prompt = f"""### Instruction:\n{prompt}\n\n### Context:\n{context}\n\n### Response:"""
    else:
        raise Exception('Invalid model name')

    encodeds = tokenizer(tokenized_prompt, return_tensors="pt", add_special_tokens=False)
    model_inputs = encodeds.to("cuda:0")

    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=512,
        top_p=0.95,  # Added. Seems to work!
        top_k=1000,  # Added
        do_sample=True)
    # print('encoded\n', generated_ids)
    decoded = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
    # print('Prompt: ', tokenized_prompt, '\n')
    output = decoded[0]
    
    # Extract the actual script (remove the prompt)
    if model_name == llama2:
        script = re.sub(r'[\s\S]*\[/INST\].*', '', output).lstrip()
        # ^ Any char followed by [/INST] followed by any non-newline char (.*)
        # The rest of the line usually says 'Great! Here's a meditation script that ...
        print(f'### PROMPT:\n{tokenized_prompt}\n\n')
    else:  # model_name == falcon7b:
        script = re.sub(r'.*### Response:', '', output)

    # If response did not end in full sentence, remove the last substring that does not end in a period
    last_period_idx = script.rfind('.')
    if last_period_idx != -1:
        script = script[:last_period_idx + 1]  # + 1 to keep the period

    # print(output)
    # print('\n\nSEPARATION')
    print('Context:\n', context, '\n')
    print('Transcript\n', script)

#### Setting parameters

Change MED_TYPE to any key in the `goals` dictionary. If you'd like, you can add 'goals' as the values to incorporate into the prompt. You can also leave them empty and it should run fine.  
Also you can set CONTEXT to a string that a user would provide to our LLM. There are already some commented examples.

In [5]:
GOALS = {
    'mindful observation': 'emphasizes the practice of mindfully observing oneself in a state of stillness, which includes lying meditation, sitting in silence, and observing thoughts and emotions.',
    'body-centered meditation': 'focuses on heightened awareness of the body, breath, and sensory perceptions, sometimes involving concentration on specific body locations or energy centers.',
    'visual concentration': 'involves techniques like visualizations and concentration on visual objects to achieve a centered state of mind.',
    'contemplation': 'encourages deep reflection on contradictions, paradoxes, or spiritual questions.',
    'affect-centered meditation': 'encompasses practices that cultivate positive emotions such as compassion, loving-kindness, and equanimity.',
    'mantra meditation': 'involves the repetitive use of syllables, words, or phrases, often with sounds and mantras, to facilitate meditation.',
    'movement meditation': 'combines mindfulness with physical activity, including manipulating the breath, walking, and sensory observation.'
}

#### Model falcon7b

In [6]:
model_name = falcon7b
model, tokenizer = load_model(model_name)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
med_type = 'mindful observation'

context = "I'm sad about losing a lost one. I need comfort"  # Nice: I noticed llama did address the loss
context += "\nMake sure the response ends with a complete sentence."

predict_model(model, tokenizer, med_type, context)

Context:
 I'm sad about losing a lost one. I need comfort
Make sure the response ends with a complete sentence. 

Transcript
 ### Instruction:
Write me a mindful observation meditation script which emphasizes the practice of mindfully observing oneself in a state of stillness, which includes lying meditation, sitting in silence, and observing thoughts and emotions.

### Context:
I'm sad about losing a lost one. I need comfort
Make sure the response ends with a complete sentence.


As you sit in stillness, mindfully observe the sadness and grief in your heart. Let those feelings be held gently, knowing they are valid and part of the healing process. Hold on to the sweet memories and the many ways you found to share love and gratitude with your lost one. Give yourself permission to heal and process these emotions over time.


#### Model llama2

In [6]:
model_name = llama2
model, tokenizer = load_model(model_name)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
med_type = 'mindful observation'

context = "I'm sad about losing a lost one. I need comfort"  # Nice: I noticed llama did address the loss
context += "\nMake sure the response ends with a complete sentence."

predict_model(model, tokenizer, med_type, context)

/home/javiferfer/anaconda3/lib/python3.9/site-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(


In [8]:
# NOTE: SOMETIMES. LLAMA 2 ends its reponse mid sentence. Not sure how to fix this

#### Example LLama 2 with context. personalized response to context
output

[INST] <\<SYS>>
I'm sad about losing a lost one. I need comfort
Make sure the response ends with a complete sentence.
<\</SYS>>

Write me a body-scan meditation script designed to relax and relieve stress by tightening and relaxing muscles. [/INST]  Of course, I'm here to help. Losing someone we love can be incredibly difficult and can leave us feeling sad, lost, and alone. It's important to remember that you are not alone in this feeling, and that many people have experienced this pain before.

Take a deep breath in through your nose, and hold it for a moment. Feel the air fill your lungs and your body. Exhale slowly through your mouth, letting go of any tension or stress you may be carrying.

Now, let's focus on relaxing your body. Starting at the crown of your head, work your way down to your toes, tensing and relaxing each muscle group as you go.

* Tense your shoulders, holding them up towards your ears. Breathe out and release the tension, letting your shoulders relax and fall back down.
* Tense your neck and jaw, letting go of any tension or stress. Breathe out and feel your jaw relax, your teeth parting slightly.
* Tense your chest and abdomen, feeling any tightness or discomfort. Breathe out and relax these areas, letting your breath flow freely.
* Tense your arms and legs, holding them in a relaxed position. Breathe out and let go of any tension, feeling your muscles relax and your body calm.
* Tense your feet and toes, feeling any tightness or discomfort. Breathe out and let go of any tension, feeling your feet relax and your toes spread apart.

As you continue to breathe, allow your body to relax and let go of any stress or tension. Imagine any worries or concerns floating away with your breath, leaving you feeling calm and relaxed.

Remember, it's okay to feel sad and it's okay to take time to grieve. Give yourself the time and space you need to process your emotions, and know that with time, the pain will lessen and you will find peace and healing once again.

Take another deep breath in through your nose, and hold it for a moment. Exhale slowly through your mouth, feeling relaxed and calm. Take one last deep breath, and let it go. Allow yourself to relax and let go of any tension, feeling the peace and calm wash over you.


#### Example Llama 2 with context
Prompt:
\<s>[INST] <\<SYS>>
I'm getting read to go to sleep I want to relax.
Make sure the response ends with a complete sentence.
<\</SYS>>

Write me a body-scan meditation script designed to relax and relieve stress by tightening and relaxing muscles. [/INST]  Great! Here's a body-scan meditation script to help you relax and relieve stress:

Find a comfortable and quiet place where you can lie down or sit comfortably. Take a few deep breaths and let's begin.

Close your eyes and bring your attention to your toes. As you exhale, gently squeeze your toes and hold for a few seconds. As you inhale, allow your toes to relax and let go of any tension.

Now, bring your attention to your feet. As you exhale, tighten your feet, making sure there is no tension in your toes or heels. Hold for a few seconds, and then release as you inhale. Allow your feet to relax and feel the ground beneath you.

Moving up your body, bring your attention to your ankles. As you exhale, gently squeeze your ankles and hold for a few seconds. As you inhale, allow your ankles to relax and feel the weight of your body on the ground.

Continue this process, scanning your body from your ankles to your calves, knees, thighs, hips, back, shoulders, arms, hands, and finally, your head. With each scan, take a few deep breaths and allow the muscles in that area to relax and let go of any tension.

As you exhale, imagine any stress or tension leaving your body and being replaced with a sense of calm and relaxation. Allow yourself to let go of any thoughts or worries, and simply focus on the sensation of your breath.

Take a few more deep breaths and allow yourself to fully relax. Feel the weight of your body on the ground or in your bed, and know that you are safe and supported. As you drift off to sleep, hold the image of relaxation and calm in your mind, and allow yourself to fall into a peaceful and restful slumber.

Sweet dreams!\</s>

#### Example Llama 2 with context (active)
Context:  I'm tired and sleepy in the morning and I need to wake up for the day.

 Make sure the response ends with a complete sentence

Find a comfortable seated position, either on a cushion on the floor or in a chair. Close your eyes and take a few deep breaths.

Beginning at the crown of your head, bring your attention to each muscle group in your body, starting at the top of your head and working your way down to your toes. As you focus on each area, take a deep breath in and exhale slowly, allowing any tension or stress to release.

As you continue to breathe deeply, tighten the muscles in your face, including your eyebrows, cheeks, and jaw. Hold for a few seconds, and then release and feel the relaxation spread through your face.

Next, focus on your shoulders and neck. Tighten the muscles in your shoulders, holding for a few seconds before releasing. As you exhale, feel any tension or stress melt away.

Now, move down to your arms and hands. Tighten your biceps, holding for a few seconds before releasing. As you exhale, feel any tension or stress leave your arms and hands.

Continue this process, working your way down your body, tightening and relaxing each muscle group in turn. As you focus on each area, allow any tension or stress to release and feel the relaxation spread through your body.

Take a few more deep breaths, feeling your body become more and more relaxed with each exhalation. Allow yourself to fully relax and let go of any remaining tension or stress.

When you're ready, slowly open your eyes and take a few more deep breaths, feeling refreshed and renewed.

Remember, this is just a starting point, and you can always modify the script to suit your needs and preferences.

Take a few more deep breaths, feeling your body become more and more relaxed with each exhalation. Allow yourself to fully relax and let go of any remaining tension or stress.

When you're ready, slowly open your eyes and take a few more deep breaths, feeling refreshed and renewed.

Remember, this is just a starting point, and you can always modify the script to suit your needs and preferences.

#### Example Llama 2 without context
\<s>[INST] <\<SYS>>


 Make sure the response ends with a complete sentence
<\</SYS>>

Write me a body-scan meditation script designed to relax and relieve stress by tightening and relaxing muscles. [/INST]  Sure, here is a body-scan meditation script designed to relax and relieve stress by tightening and relaxing muscles:

Find a comfortable seated position, either on a cushion on the floor or in a chair. Close your eyes and take a few deep breaths. Feel the breath move in and out of your body, allowing it to relax and let go of any tension.

Beginning at the crown of your head, bring your attention to each muscle group in your body, starting at the top and working your way down to your toes. As you focus on each area, take a deep breath in and exhale slowly, allowing any tension to release.

In your neck and shoulders, feel the muscles relax and release as you breathe. Imagine any knots or tension melting away with each exhalation.

As you move down to your upper back, chest, and arms, feel the muscles relax and lengthen, allowing your body to open and expand.

In your abdomen, feel the muscles soften and release, allowing your body to relax and let go of any tension.

As you move down to your lower back, hips, and legs, feel the muscles relax and release, allowing your body to move freely and easily.

In your feet and toes, feel the grounding sensation of the earth, and imagine any remaining tension melting away as you exhale.

Take a few more deep breaths, feeling the relaxation and release throughout your body. When you are ready, slowly open your eyes and take a moment to notice how you feel.

Remember, the more you practice body-scan meditation, the more you will become aware of your body and the ways in which it can relax and release tension. Take a few moments each day to come back to this practice and allow yourself to feel more relaxed and at ease in your body.